**Step 1**
Note : Skip if not need 

only run when google drive need to mount drive
Mount Google drive to retrive data, save model weight and save results

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Step 2 :** 

TPU model only avalible tensorflow 1.13.1 before version

In [0]:
pip install tensorflow==1.13.1

**Step 3 :** 

Import nessacery pacege for code

In [0]:
# To run and train model use below command 
# python3 train_model_ml.py  '-m' 'sl' '-e' '10' '-b' '256' '-alpha' '0.1' '-beta' '1.0' '-c' 'C5'

from __future__ import absolute_import
from __future__ import print_function
import os
import time
import numpy as np
import pandas as pd
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input,Reshape, Conv1D, Dense, MaxPooling1D, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, layers
from tensorflow.keras.regularizers import l2
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping, TensorBoard, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback, LearningRateScheduler

**Step 4:**

Assign Dataset name to A or B to run model on that dataset

Set path for train and test dataset

In [0]:
dataset_name = 'A'
# drive_path_main_folder = '/content/drive/My Drive/ML_Project'
X_Train_Data_Sample_Path = r'%s_train_samples.xlsx'% (dataset_name)
Y_Train_Data_label_Path = r'%s_train_labels.xlsx'% (dataset_name)

X_Test_Data_Loaded_Path = r'%s_test_samples.xlsx'% (dataset_name)
Y_Test_Data_label_Path = r'%s_test_labels.xlsx'% (dataset_name)

**Step 5 :**

Read data from excel to local variable to use it future for different column

In [0]:
X_Train_data_loaded = pd.read_excel(X_Train_Data_Sample_Path)
Y_Train_data_loaded = pd.read_excel(Y_Train_Data_label_Path)
X_Test_Data_Loaded = pd.read_excel(X_Test_Data_Loaded_Path)
Y_Test_data_loaded = pd.read_excel(Y_Test_Data_label_Path)

**Step 6 :**

Set parameter like column you want to run, batch size, number of epoch, alpha, beta (for loss function), split percentage for validation and train

In [0]:
column_name = 'B'
num_classes = Y_Test_data_loaded['Labels'].max()
model_name ='sl'
batch_size = 256
if dataset_name == "A":
  epochs = 40
else: 
  epochs = 20
split_per = 0.2
num_of_counter = 4
alpha = 0.01
beta = 0.1

**Step 7 :**

Data proeccsing using Selected column for lable and formate dataset to run on tpu model

Train Sample data proecssing:


1.   Find Mean for X train samples 
2.   Substract mean from X train samples
3.   Transfrom data between 0 to 1 using MinMaxScaler
4.   Split data for Validation
5. Transfrom lable to 0 and 1 using to_categorical
6. Reshape X train sample (because model accept that way
7. Repet Step 2 to 6 for test data sample and lable 




In [0]:
Xm = X_Train_data_loaded
Y = Y_Train_data_loaded-1
means = Xm.mean(axis=0)
Xm = (Xm - means)
x_scaler = MinMaxScaler()
X_train = x_scaler.fit_transform(Xm)
X_train, x_validation, y_train, y_validation = train_test_split(X_train,Y,test_size = split_per,train_size = 1 - split_per)

y_train = pd.DataFrame(y_train[column_name])
y_train = np_utils.to_categorical(y_train,num_classes)

y_validation = pd.DataFrame(y_validation["B"]).values
y_validation = np_utils.to_categorical(y_validation,num_classes)

input_features = X_train.shape[1]
seq_len = X_train.shape[0]

X_train = np.array(X_train).reshape(seq_len, input_features, 1)
x_validation = np.array(x_validation).reshape(x_validation.shape[0], input_features, 1)

X_T = X_Test_Data_Loaded - means
Y_T = Y_Test_data_loaded['Labels'].values-1

X_test = x_scaler.transform(X_T)
y_test = np_utils.to_categorical(Y_T,num_classes)
X_test = np.array(X_test).reshape(X_test.shape[0], input_features, 1)

**Step 8:** 

Build class for logging data while traing model also, track and save accuracy and loss while running code

also class for tracking SGD learing rate

In [0]:

class LoggerCallback(Callback):
    """
    Log train/val loss and acc into file for later plots.
    """
    def __init__(self, model, X_train, y_train,  model_name, epochs, alpha, beta):
        super(LoggerCallback, self).__init__()
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.n_class = y_train.shape[1]
        self.model_name = model_name
        self.epochs = epochs
        self.alpha = alpha
        self.beta = beta

        self.train_loss = []
        self.test_loss = []
        self.train_acc = []
        self.test_acc = []
        self.train_loss_class = [None]*self.n_class
        self.train_acc_class = [None]*self.n_class

        # the followings are used to estimate LID
        self.lid_k = 20
        self.lid_subset = 128
        self.lids = []

        # complexity - Critical Sample Ratio (csr)
        self.csr_subset = 500
        self.csr_batchsize = 100
        self.csrs = []

    def on_epoch_end(self, epoch, logs={}):
        tr_acc = logs.get('acc')
        tr_loss = logs.get('loss')
        val_loss = logs.get('val_loss')
        val_acc = logs.get('val_acc')

        self.train_loss.append(tr_loss)
        self.test_loss.append(val_loss)
        self.train_acc.append(tr_acc)
        self.test_acc.append(val_acc)

        print('ALL acc:', self.test_acc)

        file_name = 'loss_%s_%s' % (self.model_name, self.alpha)
        np.save(file_name, np.stack((np.array(self.train_loss), np.array(self.test_loss))))
        file_name = 'acc_%s_%s' % (self.model_name, self.alpha)
        np.save(file_name, np.stack((np.array(self.train_acc), np.array(self.test_acc))))

        return
        
class SGDLearningRateTracker(Callback):
    def __init__(self, model):
        super(SGDLearningRateTracker, self).__init__()
        self.model = model

    def on_epoch_begin(self, epoch, logs={}):
        init_lr = float(K.get_value(self.model.optimizer.lr))
        decay = float(K.get_value(self.model.optimizer.decay))
        iterations = float(K.get_value(self.model.optimizer.iterations))
        lr = init_lr * (1. / (1. + decay * iterations))
        print('init lr: %.4f, current lr: %.4f, decay: %.4f, iterations: %s' % (init_lr, lr, decay, iterations))

**Step 9:** 

Symantric cross entropy for model
which goal of this project using this loss function train model with noisy lable data

In [0]:
def symmetric_cross_entropy(alpha, beta):
    def loss(y_true, y_pred):
        y_true_1 = y_true
        y_pred_1 = y_pred

        y_true_2 = y_true
        y_pred_2 = y_pred

        y_pred_1 = tf.clip_by_value(y_pred_1, 1e-7, 1.0)
        y_true_2 = tf.clip_by_value(y_true_2, 1e-4, 1.0)

        return alpha*tf.reduce_mean(-tf.reduce_sum(y_true_1 * tf.math.log(y_pred_1), axis = -1)) + beta*tf.reduce_mean(-tf.reduce_sum(y_pred_2 * tf.math.log(y_true_2), axis = -1))
    return loss

**Step 10:**

TPU model require data in batch size so it can achive parraller processing while training model

In [0]:
def train_input_fn():
    # convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float32),y_train))
    
    dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)

    # return the dataset.
    return dataset

**Step 11:**

Decrease learing rate based on epoch so it can learn better

In [0]:
def get_lr_scheduler():
    def scheduler(epoch):
        if epoch < 20:
            return 0.2
        elif epoch < 35:
            return 0.15
        else:
          return 0.1
    return LearningRateScheduler(scheduler)

**Step 12:**

Build model, Compile model, set callback for tracking, convert model to TPU model and train TPU model on dataset

In [0]:
def train():
    print("seq_len", seq_len, "num_classes", num_classes, "input_features:", input_features)
    
    # load model
    model = get_model(num_classes)
    model.summary()
    
    # Select Optimizer
    optimizer = SGD(lr=0.15, decay=1e-4, momentum=0.9,nesterov=True)

    # create loss
    loss = symmetric_cross_entropy(alpha,beta)

    # model
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=['accuracy']
    )

    model_save_file = "model_%s_{epoch:02d}_%s.hdf5" % (model_name, column_name)
    
    ## do real-time updates using callbakcs
    callbacks = []

    cp_callback = ModelCheckpoint(model_save_file,
                                    monitor='val_loss',
                                    verbose=0,
                                    save_best_only=True,
                                    save_weights_only=True,
                                    period=1)
    callbacks.append(cp_callback)
    
    # learning rate scheduler if use sgd
    lr_scheduler = get_lr_scheduler()
    callbacks.append(lr_scheduler)
    callbacks.append(SGDLearningRateTracker(model))
    # learning rate scheduler if use sgd
    lr_scheduler = get_lr_scheduler()
    callbacks.append(lr_scheduler)

    # acc, loss, lid
    log_callback = LoggerCallback(model, X_train, y_train, model_name, epochs, alpha, beta)
    callbacks.append(log_callback)
    callback_early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
    callbacks.append(callback_early_stopping)

    # train model
    try:
        device_name = os.environ['COLAB_TPU_ADDR']
        TPU_ADDRESS = 'grpc://' + device_name
        print('Found TPU at: {}'.format(TPU_ADDRESS))
    except KeyError:
        print('TPU not found')
  
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(
        model,
        strategy=tf.contrib.tpu.TPUDistributionStrategy(
            tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS))
    )

    tpu_model.fit(train_input_fn,
                    epochs=epochs,
                    steps_per_epoch=int(len(X_train) / batch_size),
                    validation_data = (x_validation, y_validation), 
                    validation_steps =int(len(x_validation) / batch_size),
                    callbacks=callbacks)

**Step 13:**

Build model, load weight from save model weight and then test model on test dataset

In [0]:
def test(need_to_predict = False):
    seq_len = X_test.shape[0]
    # load model
    model = get_model(num_classes)
    
    # Select Optimizer
    optimizer = SGD(lr=0.1, decay=1e-4, momentum=0.9,nesterov=True)

    # create loss
    loss = symmetric_cross_entropy(alpha, beta)
    
    # model
    model.load_weights(save_model_path)
    
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=['accuracy']
    )
    score = model.evaluate(X_test.astype(np.float32), y_test, verbose=0)
    print("%s: %.2f%%" %(model.metrics_names[1], score[1]*100))
    print("%s: %.2f%%" %(model.metrics_names[0], score[0]*100))

    if need_to_predict:
      y_pred = model.predict(X_test.astype(np.float32))

      y_pred=np.argmax(y_pred, axis = 1)
      y_test_=np.argmax(y_test, axis = 1)
      f1_score_mesure = f1_score(y_test_, y_pred, average = 'weighted')

      print("F measure ", f1_score_mesure)
      return y_pred, f1_score_mesure, score

**Step 14:**

Build model using Conv1D and 3 hidden layer

In [0]:
def get_model(num_classes=10):
    
    inputs_data = Input(shape=(input_features,1))
    b = Conv1D(3, 3, padding='same', kernel_initializer="he_normal", name='block1_conv1')(inputs_data)
    b = BatchNormalization()(b)
    b = Activation('relu')(b)

    b = Conv1D(3, 3, padding='same', kernel_initializer="he_normal", name='block1_conv2')(b)
    b = BatchNormalization()(b)
    b = Activation('relu')(b)

    b = MaxPooling1D(2, strides=2, name='block1_pool')(b)

    b = Flatten(name='flatten')(b)

    b = Dense(128)(b)
    b = Activation('relu',name='lid')(b)
    b = Dropout(0.2)(b)
   
    b = Dense(32)(b)
    b = Activation('relu')(b)
    b = Dropout(0.2)(b)

    b = Dense(num_classes)(b)
    outputs_lable = Activation('softmax')(b)
    model = Model(inputs=inputs_data, outputs=outputs_lable)
    return model

**Step 15:**

Train model, test model and see the results, predict value by train model

In [0]:
train()
need_to_predict = True
y_pred_obj = {}
save_data_file_name = time.strftime("%Y%m%d-%H%M%S")
Save_File_Name = '%s_data_%s.csv' % (column_name, save_data_file_name)
Save_File_Name_Text = 'Text_%s_data_%s.txt' % (column_name, save_data_file_name)
counter = num_of_counter
text_file = open(Save_File_Name_Text,"w+")
for x in range(epochs):
  x = epochs - x
  if counter < 0:
    break
  if x < 10:
    save_model = "%s_%s%s_%s" % (model_name,0,x, column_name)
  else:
    save_model = "%s_%s_%s" % (model_name,x, column_name)
  save_model_path = "model_%s.hdf5" % (save_model)
  if os.path.exists(save_model_path):
    counter = counter - 1
    print(save_model_path)
    if need_to_predict:
      y_pred, f1_score_mesure, score = test(True)
      text_file.write("Accuracy %s\r\n\n F measure %s\r\n\n Score %s\r\n\n\n\n\n" % (score[1]*100, f1_score_mesure,score ))
      y_pred_obj.update( {save_model : y_pred + 1} )
    else:
      test()

if need_to_predict:
  text_file.close()
  df = pd.DataFrame(y_pred_obj)
  df.to_csv(Save_File_Name,index=False) 

**Step 16:**

Just For Understaing
this will return error in each column compare to B

In [0]:
column_array = ['B', 'C5', 'C10', 'C15', 'C20', 'C25', 'C30', 'C35', 'C40', 'A5', 'A10', 'A15', 'A20', 'A25', 'A30', 'A35', 'A40', 'N5', 'N10', 'N15', 'N20', 'N25', 'N30', 'N35', 'N40']
for i in range(len(column_array) -1):
  tempCount = i + 1
  tempDifference = np.where((Y_Train_data_loaded['B'] != Y_Train_data_loaded[column_array[tempCount]]), 1, 0)
  pddf = pd.DataFrame(tempDifference)
  cdcs = pddf[0].values
  print(column_array[tempCount] , (pddf[pddf.values == 1].shape[0]/ cdcs.shape[0])*100)